In [1]:
"""
Evaluate linear combination of Legendre polynomials L_k(x) given in point x from 1d interval.
In short, f(x) = sum_k c_k L_k(x) is computed.
"""
function legval{T<:Real}(x::T, c::Vector{T})
    if length(c) == 1
        c0 = c[1]
        c1 = 0.
    elseif length(c) == 2
        c0 = c[1]
        c1 = c[2]
    else
        nd = length(c)
        c0 = c[end-1]
        c1 = c[end]

        for i in range(2, nd-2)
            tmp = c0
            nd = nd - 1
            c0 = c[end-i] - (c1*(nd - 1))/nd
            c1 = tmp + (c1*x*(2*nd - 1))/nd
        end  
      end
      return c0 + c1*x
end

"""
Compute m-th derivative of Legendre series as another Legendre series.
So legval(x, legder(c, 1)) is the value at x of sum_k c_k L'_k.
"""
function legder(c::Vector{Float64}, m::Int64)
    assert(m >= 0)
    # 0 order derivative
    if m == 0
        return c
    # len(c) is 0, 1, 2, 3 degree. Check for derivative of order larger than max
    # degree
    elseif m >= length(c)
        return [0.]
    end
    
    n = length(c)
    for i in range(1, m)
        n = n - 1
        der = zeros(n)
        c = copy(c)
     
        for j in n:-1:3
            der[j] = (2*j - 1)*c[j+1]
            c[j - 1] += c[j+1]
        end
        if n > 1
            der[2] = 3*c[3]
        end
        der[1] = c[2]
        c = der
    end
    return c
end

"""
Vectorized version of legval. X is a vector of x coordinates.
"""
function legval{T<:Real}(x::Vector{T}, c::Vector{T})
    y = similar(x)
    for (i, xi) in enumerate(x)
        y[i] = legval(xi, c)
    end
    y
end

legval (generic function with 2 methods)

In [2]:
# Test against sympy
c = [1., 2., 3., -1.]
points = [0.2, 0.1, 0.5, 0.6]

values = legval(points, c)
for (x, v) in zip(points, values)
    println("x=$(x) ", 
            "f=$(v) ",
            "df=$(legval(x, legder(c, 1))) ",
            "d^2f=$(legval(x, legder(c, 2)))")
end

x=0.2 f=0.3600000000000001 

In [3]:
"""
Evaluate linear combination of Legendre polynomials L_i(x)*L_j(y) given in point x from 2d.
The polynomials are obtained as tensor product of 1d polynomial in x and y, i.e.
P(x, y) = c[0, 0]*L_0(x)*L_0(x) + c[0, 1]*L_0(x)*L_1(y) ...
"""
function legval{T<:Real}(x::T, y::T, c::Matrix{T})
    # First eval at x coord polynomials of x specified by columns in c
    ncols = size(c, 2)
    c_x = zeros(T, ncols)
    for col in 1:ncols
        c_x[col] = legval(x, c[:, col])
    end
    # Now eval at y with new coefs
    legval(y, c_x)
end

"""
Compute derivatives of 2d(tensor product) Legendre series. The derivative is specified
by a multidex m = [mx, my] and requests mx derivative w.r.t to x and my derivative w.r.t
y.
"""
function legder{T<:Real}(c::Matrix{T}, m::Vector{Int64})
    assert(all(v -> v >= 0, m))   # No antiderivativec
    assert(length(m) == 2)        # Multiindex of length 2
    # Zero-th order derivative
    if m == [0, 0]
        return c
    # Taking derivatives higher than the degree
    elseif m[1] >= size(c, 1) || m[2] >= size(c, 2)
        return zeros(1, 1)
    end
    
    # Compute
    # Partial derivative w.r.t x
    if m == [1, 0]        
        nrows, ncols = size(c, 1), size(c, 2)
        # Take derivative col by col
        diff = zeros(nrows-1, ncols)
        for col in 1:ncols
            diff[:, col] = legder(c[:, col], 1)
        end
        return diff
    # Partial derivative w.r.t y 
    elseif m == [0, 1]
        # Clever use of recursion :)
        return legder(c', [1, 0])'
    # Higher order stuff
    else
        mx, my = first(m), last(m)
        coef = copy(c)
        # We take the sweps in x and y. This should be more stable then loweing 
        # x and then y.
        while mx > 0 || my > 0
            # \partial x
            if mx > 0
                coef = legder(coef, [1, 0])
                mx -= 1
            end
            # \partial y
            if my > 0
                coef = legder(coef, [0, 1])
                my -= 1
            end
        end
        return coef
    end
end

"""
Vectorized version of legval. X is a (number of points, 2) matrix of coordinates.
"""
function legval{T<:Real}(x::Matrix{T}, c::Matrix{T})
    nrows = size(x, 1)
    y = zeros(T, nrows)
    for i in 1:nrows
        y[i] = legval(x[i, 1], x[i, 2], c)
    end
    y
end

legval (generic function with 4 methods)

In [4]:
# Test against sympy
c = [1. 2. -1; 3 4 0.2]
points = [0.3 0.4; -0.2 0.6; 0.8 -1.]


f = legval(points, c)
dx = legval(points, legder(c, [1, 0]))
dy = legval(points, legder(c, [0, 1]))
dxdx = legval(points, legder(c, [2, 0]))
dxdy = legval(points, legder(c, [1, 1]))
dydy = legval(points, legder(c, [0, 2]))
dydydy = legval(points, legder(c, [0, 3]))

println(f)
println(dx)
println(dy)
println(dxdx)
println(dxdy)
println(dydy)
println(dydydy)

df=5.0 d^2f=6.0
x=0.1 f=-0.10750000000000004 df=4.325 d^2f=7.5
x=0.5 f=2.0625 df=6.125 d^2f=1.5
x=0.6 f=2.68 df=6.199999999999999 d^2f=0.0
[

In [5]:
typealias MatVec Union{Vector, Matrix}

# Legendre polynomial is given by expansion coefs w.r.t basis of L_k
type LegendrePolynomial
    coefs::MatVec
end

# Evaluate legendre polynomial: vec[vec], mat[mat]
function Base.call(f::LegendrePolynomial, x::MatVec)
    legval(x, f.coefs)
end

evaluate(f::LegendrePolynomial, x::MatVec) = f(x)

# Compute derivivative specified by multi-index
function evaluate_derivate(f::LegendrePolynomial, m::Vector{Int64}, x::MatVec)
    if length(m) == 1
        return legval(x, legder(f.coefs, m[1]))
    else
        println(m, typeof(f.coefs), typeof(m))
        return legval(x, legder(f.coefs, m))
    end
end

evaluate_derivate (generic function with 1 method)

In [6]:
c = [1., 2., 3., -1.]
points = [0.2, 0.1, 0.5, 0.6]

f = LegendrePolynomial(c)
evaluate(f, points)
evaluate_derivate(f, [1], points)

4-element Array{Float64,1}:
 5.0  
 4.325
 6.125
 6.2  

3.4244000000000003,1.0784,-2.6399999999999997]
[4.548,5.4079999999999995,-0.8000000000000003]
[2.072,-0.6719999999999999,7.720000000000001]
[0.0,0.0,0.0]
[4.24,4.36,3.4]
[-2.82,-3.12,-2.52]
[0.0,0.0,0.0]


In [7]:
c = [1. 2. -1; 3 4 0.2]
points = [0.3 0.4; -0.2 0.6; 0.8 -1.]

f = LegendrePolynomial(c)
fvals = evaluate(f, points)
dfvals = evaluate_derivate(f, [1, 0], points)

[1

3-element Array{Float64,1}:
  4.548
  5.408
 -0.8  

In [8]:
# Just try make basis in 1d
typealias LegendreBasis Vector{LegendrePolynomial}

# The dimension will be indicated by cell type
type Line
end

# The constructor which takes in matrix whose columns are the coeficients
function LegendreBasis(coefs::Matrix)
    assert(abs(det(coefs)) > 1E-14)
    LegendreBasis([LegendrePolynomial(coefs[:, col]) for col in 1:size(coefs, 2)])
end

# If degree is given, we use columns of identity matrix
LegendreBasis(line::Line, degree::Int64) = LegendreBasis(eye(degree))

Array{LegendrePolynomial,1}

In [9]:
# don't forget immutable!
# Function, simple algebraic operations...
#

abstract LinearFunctional

type PointEvaluation <: LinearFunctional
    point::Vector
    id::UInt64
    
    function PointEvaluation(p::Vector)
        id = hash((p, 0))
        new(p, id)
    end
end

Base.call(l::PointEvaluation, f::Function) = f(l.point)



PointEvaluation([1.]).id

,0]Array{Float64,2}Array{Int64,1}


0x881421628c7aad7b

In [66]:
# Stuff for quadrature
"""Points and weight for exact integration of polynomials of degree = 2*degree -1."""
function gauss_legendre(domain::Line, degree::Int)
    #The implementation is based on eigenvalues of the matrix T that is similar to
    #the Jacobi matrix J. Both are tridiagonal but the T matrix is symmetric and
    #the main diagonal is 0.
    diag = zeros(degree)
    up_diag = Float64[k/sqrt(4*k^2-1) for k=1:degree-1]
    T = SymTridiagonal(diag, up_diag)
    xq, ev = eig(T)
    wq = reshape(2*ev[1, :].^2, degree)
    return xq, wq
end

# Stuff for 2d
function gauss_legendre(domain::Line, degree::Tuple{Int64, Int64})
    degree_x, degree_y = first(degree), last(degree)
    xq, wq = gauss_legendre(domain, degree_x)
    if degree_x == degree_y
        weights = Float64[i*j for i in wq, j in wq]
        points = hcat([Vector{Float64}([x, y]) for x in xq, y in xq])
    end
    collect(points), collect(weights)
end

gauss_legendre (generic function with 3 methods)

In [70]:
xq, wq = gauss_legendre(Line(), (3, 3))

([[-0.7745966692414824,-0.7745966692414824],[8.881784197001252e-16,-0.7745966692414824],[0.7745966692414834,-0.7745966692414824],[-0.7745966692414824,8.881784197001252e-16],[8.881784197001252e-16,8.881784197001252e-16],[0.7745966692414834,8.881784197001252e-16],[-0.7745966692414824,0.7745966692414834],[8.881784197001252e-16,0.7745966692414834],[0.7745966692414834,0.7745966692414834]],[0.3086419753086436,0.49382716049382724,0.3086419753086429,0.49382716049382724,0.7901234567901196,0.4938271604938262,0.3086419753086429,0.4938271604938262,0.30864197530864224])

In [71]:
wq

9-element Array{Float64,1}:
 0.308642
 0.493827
 0.308642
 0.493827
 0.790123
 0.493827
 0.308642
 0.493827
 0.308642

In [12]:
line = Line()
basis = LegendreBasis(line, 10)
xq, wq = gauss_legendre(line, 11)

M = zeros(10, 10)
for (i, u) in enumerate(basis), (j, v) in enumerate(basis)
    M[i, j] = sum(wq.*(u(xq).*v(xq)))
end

M0 = diagm([2/(2k-1) for k in 1:10])

norm(M-M0)

2.5793354566071804e-14

In [13]:
# AbstractFunction (polynomial degree)
# AbstractFunctional
# AbstractCell
#

In [34]:
x = [1, 2, 3]
y = [4, 5, 6]
[(xi, yi) for xi in x, yi in y]

3x3 Array{Tuple{Any,Any},2}:
 (1,4)  (1,5)  (1,6)
 (2,4)  (2,5)  (2,6)
 (3,4)  (3,5)  (3,6)

In [35]:
hcat(x, y)

3x2 Array{Int64,2}:
 1  4
 2  5
 3  6

In [16]:
a = (1, )

(1,)

In [17]:
length(a)

1

In [36]:
collect(x, y)

LoadError: LoadError: MethodError: `collect` has no method matching collect(::Array{Int64,1}, ::Array{Int64,1})
Closest candidates are:
  collect{T}(!Matched::Type{T}, ::Any)
  collect(::Any)
while loading In[36], in expression starting on line 1